## Дополнение к семинарам 2 и 3. Пояснение нескольких важных концепций numpy и pandas

In [12]:
import numpy as np
import pandas as pd

### Как и зачем применять логические функции к массивам?

В python, как мы знаем, три стандартных логических операции обозначаются and, or, not, и используются они, например, в if-блоках. Первые две операции являются бинарными, а последняя — унарная.

In [3]:
print True and False, True or False, not True
x = 5
print x > 2 and x < 10, x < 2 or x > 10, not x > 2 

False True False
True False False


Логические операции можно применять поэлементно и к булевым массивам:

In [6]:
A = np.random.randint(2, size=20).reshape(4, 5).astype(bool)  # метод astype изменяет тип объекта
B = np.random.randint(2, size=20).reshape(4, 5).astype(bool)
print A
print B
print np.logical_and(A, B)  # бинарная операция, два аргумента
print np.logical_or(A, B)   # бинарная операция, два аргумента
print np.logical_not(A)     # унарная операция, один аргумент

[[False False  True False False]
 [ True  True False False  True]
 [ True False  True False  True]
 [False False  True False  True]]
[[ True False  True  True  True]
 [False  True  True  True  True]
 [False False False False  True]
 [False  True False False False]]
[[False False  True False False]
 [False  True False False  True]
 [False False False False  True]
 [False False False False False]]
[[ True False  True  True  True]
 [ True  True  True  True  True]
 [ True False  True False  True]
 [False  True  True False  True]]
[[ True  True False  True  True]
 [False False  True  True False]
 [False  True False  True False]
 [ True  True False  True False]]


Логические массивы используются, например, для логической индексации (об этом ниже). 

Но иногда нужно использовать логические операции над матрицами, например, в условных блоках (произвести логические операции с матрицами и получить одно булево значение, то есть узнать, удовлетворяют ли массивы какому-то критерию). Для этого есть функции:

* np.all: возвращает True, если и только если *все* элементы равна True
* np.any: возвращает True, если и только если в массиве есть *хотя бы один* элемент, равный True

Аналогично суммированию и произведению эти операции можно применять по какой-то оси, указывая параметр функции axis=... .

Любители дискретной математики и формальных опредений могут смотреть на эти операции так:

np.all(A) = $\&_{i, j} A[i, j]$

np.any(A) = $V_{i, j} A[i, j]$

In [5]:
# Пример
A = np.array([[True, False], [True, False]])
B = np.array([[True, True], [True, True]])
C = np.array([[1, 2], [1, 2]])
D = C + 5
F = np.array([[1, 1], [1, 1]])
print np.all(A), np.all(B), np.all(A, axis=0), np.all(B, axis=0)
print np.any(A), np.any(B), np.any(A, axis=1), np.any(B, axis=1)
print np.all(C < D)
print np.all(C == F), np.any(C == F)

False True [ True False] [ True  True]
True True [ True  True] [ True  True]
True
False True


Таким образом,  np.logical_and, np.logical_or и np.logical_not — поэлементные функции (не меняют размер входных массивов),
np.all и np.any — агрегирующие функции (вообще говоря, либо уменьшают размер массива, либо просто возвращают скалярное значение).

### Зачем нужны мнимые оси (np.newaxis)?

Мнимые оси нужны, чтобы можно было выполнять операции над массивами, имеющими разный размер, то есть shape.

В numpy много поэлементных операций между двумя массивами. 

Единственное условие — чтобы массивы имели *почти одинаковый размер*. То есть если мы хотим выполнить поэлементную операцию между массивами A и B, то:
* Кортежи (tuples) A.shape и B.shape должны быть одинаковой длины. 
* Пусть A.shape = $(x_1, \dots, x_n)$, а B.shape = $(y_1, \dots, y_n)$. Тогда либо $x_i = y_i$, либо одно из этих значений равно 1. В последнем случае при выполнении операции numpy «мысленно» размножит (много раз продублирует) массив по этой размерности, и операция выполнится так, как будто массивы имели одинаковый размер.

Соответственно, с помощью np.newaxis любые два массива можно привести к *почти одинаковому размеру*. Главное, чтобы это было осмысленно и решало поставленную задачу :)

In [7]:
# Пример:
A = np.random.randint(50, size=12).reshape(3, 4)
B = np.random.randint(50, size=20).reshape(4, 5)
print A.shape, B.shape

(3, 4) (4, 5)


In [8]:
# Построим массив, состоящий из всех пар A[i, j]*B[j, k], i=1, ..., 3, j = 1, ..., 4, k = 1, ..., 5
C = A[:, :, np.newaxis] * B[np.newaxis, :, :]
print C.shape

(3, 4, 5)


In [9]:
# Проверим:
print A[1, 2]*B[2, 4]
print C[1, 2, 4] 
# совпало

680
680


> Пояснение: Мы привели обы массива к одинаковой форме (3, 4, 5). 

>Первый массив «мысленно» повторился 5 раз по оси 2 и стал трёхмерным, а второй массив "мысленно" повторился 3 раза по оси 0 и тоже стал трёхмерным. 

> Дальше их попарно перемножили.

Конечно же, реально numpy ничего не размножает и не дублирует (это был бы очень неэкономный расход оперативной памяти), но концептуально применение мнимой оси работает именно так.

In [10]:
# Эпилог примера
# Теперь, если просуммировать C по axis=1, он, очевидно (по формуле) станет равен произведению двух матриц:
D = C.sum(axis=1)
print D.shape
print np.all(D==A.dot(B))
# Мы применили мнимые оси к решению задачи, которая и так реализована в numpy. 
# Однако часто бывают случаи, когда нужно произвести операци, которых нет в numpy, 
# и их можно сделать только с помощью мнимых осей.

(3, 5)
True


### Что такое логическая индексация и зачем она нужна?

Часто нужно произвести операцию не со всеми элементами массива, а только с некоторыми, удовлетворяющими некоторому условию. В этом случае помогает логическая индексация. Пусть A — какой-то массив и I — булев массив, A.shape == I.shape. Тогда A[i] выдаст ссылки только на те элементы A, у которых на соответствующей позиции в I стоит True.  

In [43]:
A = np.arange(10)
B = np.array([True, False]*5)
print A.shape, B.shape
print A, B
print A[B]

(10L,) (10L,)
[0 1 2 3 4 5 6 7 8 9] [ True False  True False  True False  True False  True False]
[0 2 4 6 8]


### Что такое groupby? 

*Дополнительный материал для выполнения бонусного задания.*

**Начнём с не совсем понятного объяснения.**

Функция groupby библиотеки pandas нужна, чтобы объединять данные по *некоторому критерию*, а затем применять к полученному результату *некоторые функции*. 

Эти *функции* можно условно разделить на три части:
1. Агрегирующие, которые вычисляют некоторые статистики по группе. Пример: найти наименьшее значение какого-либо ключа в группе.
2. Трансформирующие, которые изменяют или формируют некоторые значения по группе. Пример: отнормировать значения в каждой группе.
3. Фильтрующие, которые удаляют некоторые группы.

Под критерием обычно понимается равенство значений некоторого подмножества ключей, но иногда разумно использовать более сложные критерии.

**Теперь попробуем разобрать всё на примерах.**

Создадим датафрейм:

In [21]:
df = pd.DataFrame({'A': [1,2,2,1,3,3], 'B': [1,2,3,3,2,1]})
df

,A,B
0,1,1
1,2,2
2,2,3
3,1,3
4,3,2
5,3,1


**Группировка**

Функция df.groupby('A') объединит данные, которые соответствуют строкам с одинаковыми значениями по ключу 'A'. Полученный результат имеет тип:

In [44]:
type(df.groupby('A'))

pandas.core.groupby.DataFrameGroupBy

У объектов типа pandas.core.groupby.DataFrameGroupBy есть поле .groups. Это словарь, в котором ключами являются значения, по которым объеденены группы, а значениями — списки индексов, которые соответствуют этим группам:

In [45]:
df.groupby('A').groups

{1: [0, 3], 2: [1, 2], 3: [4, 5]}

Но объединять можно не только совпадению ключа.

Можно передать в качестве параметра так называемую mapping-function, которая для каждой строки будет выдавать число, по равенству значений которого данные будут объединяться в группы.

Напишем функцию, которая разобьёт данные на две группы: с чётным и нечётным значением по ключу 'A':

In [98]:
# mapping function:
def check_A_odd(index): 
    if df['A'][index] % 2:
        return 'odd A'
    return 'even A'

df.groupby(check_A_odd).groups

{'even A': [1, 2], 'odd A': [0, 3, 4, 5]}

** Агрегация**

Функция .agg нужна для применения к группировке функций. Например, вот так выглядит «построчная» сумма всех элементов в группе:

In [66]:
df.groupby(check_A_odd).agg(np.sum)

,A,B
even A,4,5
odd A,8,7


Аналогичная её запись:

In [67]:
df.groupby(check_A_odd).sum()

,A,B
even A,4,5
odd A,8,7


Вот так можно агрегировать данные по одному столбцу:

In [79]:
df.groupby(check_A_odd).agg({'B': np.sum})

,B
even A,5
odd A,7


Таким образом можно применять многие стандартные функции из numpy. Например, np.mean, np.std, np.prod и так далее.

**Трансформация** 

Для трансформации используется функция .transform, о ней мы здесь говорить не будем.

**Фильтрация** 

Для фильтрации используется метод .filter, он возвращает подмножество строк исходного датафрейма. 

Например:

In [102]:
df

,A,B
0,1,1
1,2,2
2,2,3
3,1,3
4,3,2
5,3,1


In [101]:
df.groupby('A').filter(lambda x: x['A'].sum() > 3)

,A,B
1,2,2
2,2,3
4,3,2
5,3,1


Получили строки, соответствующие группам, суммарное значение по ключу 'A' которых больше трёх.

Здесь мы использовали так называемые лямбда функции. Это просто удобный способ записать короткую функцию в одну строчку.

In [105]:
lambda x: x['A'].sum() > 3 

# Аналогичная функция:
def some_function_name(x):
    return x['A'].sum() > 3

Всего этого должно хватить для решения простых задач. 

Более подробно операция groupby, как обычно, объяснена в [документации](http://pandas.pydata.org/pandas-docs/stable/groupby.html).